# World Electricity Production
Energy production is an essential part of every functioning society. Energy production is complex and requires much effort and money to not only maintain, but expand. As communities grow, the demand grows. As communities develop and transform not only does the demand grow more, but new and existing items, equipment, tools, appliances, etc. now require energy as well. Fortunately with developing socities and technological advancements our abilities to capture and produce energy grow as well. 

In this notebook, we are going to analyze monthly electricity production data collected from the International Energy Agency (IEA). This data provides us with information on the amount of energy produced (in GWh) by a specific energy product type during a specific month and year by a country. Below is schema that layouts all the data columns provided in our dataset as well as what data types and values each column take.

#### Dataset Columns:
1. COUNTRY: Country Name
2. CODE_TIME: A code that represents the month and year (ex. JAN2010 for January 2010)
3. TIME: The month and year of the data entry
4. YEAR: The year of the data point
5. MONTH: The month of the data point
6. MONTH_NAME: The month of the data point as a string
7. PRODUCT: The type of energy product (ex. Coal, Hydro, Solar, Wind, etc.)
8. VALUE: The amount of electricity produced by the energy product in gigawatt-hours (GWh)
9. DISPLAY_ORDER: The order in which the data is displayed
10. yearToDate: The amount of energy produced for the current year up to the current month
11. previousYearToDate: The amount of energy produced for the previous year up to the current month
12. share: The share of the product in the total energy production for the country

#### Project Questions/Objectives:

 - What is the average energy produced by any country?
 - What is the average energy produced by each energy product?
 - Which countries have produced the most energy and which have produced the least amount of energy?
 - What is the minimum and maximum energy produced by a country in any given month?
 - Which years had the lowest and highest energy production and demand?
 - How does energy production and demand change from season to season?
 - How has the amount of energy produced by renewable energy products changed over time?
 - How did the COVID-19 pandemic change energy production and demand?

First, let's load in the Python libraries we need to load and perform operations on our data.

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import sqlalchemy
%load_ext sql

In [2]:
data = "world_energy_production.db"
conn = sqlite3.connect(data)

In [3]:
sqlalchemy.create_engine('sqlite:///world_energy_production.db')

Engine(sqlite:///world_energy_production.db)

In [4]:
raw_data = pd.read_csv('world_energy_production.csv')

In [5]:
raw_data.to_sql('world_energy_production', conn, if_exists='replace')

181915

Now let's connect to our database to get access to the data. To do so we will use the `%sql` extention from perform and connect to our sqlite database using `sqlite:///world_energy_production.db`.

In [6]:
%sql sqlite:///world_energy_production.db

Let's take a preview of our data, to do we are going to `SELECT` all of the columns in our `world_energy_production` table. We'll also `LIMIT` the output to 25 records.

In [7]:
%%sql
SELECT *
FROM 
    world_energy_production
LIMIT 25;

 * sqlite:///world_energy_production.db
Done.


index,COUNTRY,CODE_TIME,TIME,YEAR,MONTH,MONTH_NAME,PRODUCT,VALUE,DISPLAY_ORDER,yearToDate,previousYearToDate,share
0,Australia,JAN2010,January 2010,2010,1,January,Hydro,990.728,1,16471.891,None,0.0477707293526417
1,Australia,JAN2010,January 2010,2010,1,January,Wind,409.469,2,4940.909,None,0.0197436963296655
2,Australia,JAN2010,January 2010,2010,1,January,Solar,49.216,3,908.238,None,0.0023730874829616
3,Australia,JAN2010,January 2010,2010,1,January,Geothermal,0.083,4,0.996,None,4.002077801646127e-06
4,Australia,JAN2010,January 2010,2010,1,January,Total combustible fuels,19289.73,7,214302.969,None,0.9301084365391248
5,Australia,JAN2010,January 2010,2010,1,January,Coal,14796.776,8,163887.775,None,0.7134680574160261
6,Australia,JAN2010,January 2010,2010,1,January,Oil,732.302,9,2447.778,None,0.0353099949192899
7,Australia,JAN2010,January 2010,2010,1,January,Natural gas,3544.365,10,45678.585,None,0.1709014998485719
8,Australia,JAN2010,January 2010,2010,1,January,Combustible renewables,216.287,11,2288.826,None,0.0104288843552365
9,Australia,JAN2010,January 2010,2010,1,January,Net electricity production,20739.227,14,236625.007,None,1.0


Let's extract the number of distinct countries in our dataset. To do this we are going to use the `COUNT` and `DISTINCT` clauses.

In [8]:
%%sql
SELECT 
    COUNT(DISTINCT(COUNTRY)) AS "total_unique_countries"
FROM 
    world_energy_production;

 * sqlite:///world_energy_production.db
Done.


total_unique_countries
52


Now that we have the number of unqiue countries, let's view the countries in our data.

In [9]:
%%sql
SELECT 
    DISTINCT(COUNTRY) AS "unique_countries"
FROM 
    world_energy_production;

 * sqlite:///world_energy_production.db
Done.


unique_countries
Australia
Austria
Belgium
Canada
Chile
Czech Republic
Denmark
Estonia
Finland
France


Notice that there are data points in the country column that are aggregated data points. For example there are data values "OECD Americas", "OECD Asia Oceania", "OECD Europe", "OECD Total", etc. that represent the combined energy production within those regions.

In [10]:
%%sql
SELECT 
    DISTINCT(PRODUCT) AS "unique_energy_products"
FROM 
    world_energy_production;

 * sqlite:///world_energy_production.db
Done.


unique_energy_products
Hydro
Wind
Solar
Geothermal
Total combustible fuels
Coal
Oil
Natural gas
Combustible renewables
Net electricity production


Notice that one of the energy product types that appears in our output is "final consumption" which provides the demand of energy for that particular data point.

Additionally, there are other energy product types that are aggregates of other energy products such as "Fossil Fuels", "Renewables", "Non-renewables", etc.

To avoid any issues with our analysis that these aggregated data points may cause, I am going to create a series of views in our database. The first will be the called `net_electricity_production` that only includes the records in our data without the aggregated data points in the `Country` and `Product` data columns except for the "Net electricity production" data value in the `Product` column. Also, I am going to create a view called `energy_demand` which will only include the "final consumption" data value in the `Product` data column.

We will be using these views for the majority of our analysis.

In [11]:
%%sql
CREATE VIEW IF NOT EXISTS net_electricity_production AS
SELECT
    e.COUNTRY,
    e.YEAR,
    e.MONTH,
    e.MONTH_NAME,
    e.PRODUCT,
    e.VALUE
FROM
    world_energy_production AS e
WHERE
    e.PRODUCT = "Net electricity production"
    AND e.COUNTRY NOT LIKE '%OECD%'
    AND e.COUNTRY NOT LIKE '%IEA%';

 * sqlite:///world_energy_production.db
Done.


[]

In [12]:
%%sql
SELECT *
FROM 
    net_electricity_production
LIMIT 20;

 * sqlite:///world_energy_production.db
Done.


COUNTRY,YEAR,MONTH,MONTH_NAME,PRODUCT,VALUE
Australia,2010,1,January,Net electricity production,20739.227
Austria,2010,1,January,Net electricity production,5901.466
Belgium,2010,1,January,Net electricity production,8340.712
Canada,2010,1,January,Net electricity production,60331.621
Chile,2010,1,January,Net electricity production,4780.411
Czech Republic,2010,1,January,Net electricity production,7767.877
Denmark,2010,1,January,Net electricity production,4052.916
Estonia,2010,1,January,Net electricity production,1133.431
Finland,2010,1,January,Net electricity production,8301.546
France,2010,1,January,Net electricity production,55810.737


In [13]:
%%sql
CREATE VIEW IF NOT EXISTS energy_demand AS
SELECT
    e.COUNTRY,
    e.YEAR,
    e.MONTH,
    e.MONTH_NAME,
    e.PRODUCT,
    e.VALUE
FROM
    world_energy_production AS e
WHERE
    e.PRODUCT = "Final consumption"
    AND e.COUNTRY NOT LIKE '%OECD%'
    AND e.COUNTRY NOT LIKE '%IEA%';

 * sqlite:///world_energy_production.db
Done.


[]

In [14]:
%%sql
SELECT *
FROM energy_demand
LIMIT 10;

 * sqlite:///world_energy_production.db
Done.


COUNTRY,YEAR,MONTH,MONTH_NAME,PRODUCT,VALUE
Australia,2010,1,January,Final consumption,19317.931
Austria,2010,1,January,Final consumption,5949.988
Belgium,2010,1,January,Final consumption,8002.099
Canada,2010,1,January,Final consumption,51287.873
Chile,2010,1,January,Final consumption,4508.132
Czech Republic,2010,1,January,Final consumption,5743.855
Denmark,2010,1,January,Final consumption,3252.364
Estonia,2010,1,January,Final consumption,861.106
Finland,2010,1,January,Final consumption,9032.904
France,2010,1,January,Final consumption,51864.447


Now that we have created the two views we will be using for our analysis we can move on to the objectives of our project!

### 1. What is the average energy produced by any country?

To find the average energy produced by any country, let's first `SELECT` the and calculate the average(`AVG`) of the `VALUE` column in our table. We will be the using the `net_electricity_production` we created before to answer our question.

In [15]:
%%sql
SELECT 
    AVG(VALUE) AS "Average Energy Produced by Any Country"
FROM 
    net_electricity_production;

 * sqlite:///world_energy_production.db
Done.


Average Energy Produced by Any Country
23807.05043906806


Our output indicates that the average energy produced 23807.05 Gwh. To get a deeper more detailed understanding of the average energy produced, let's calculate the average energy produced by each country. We can do this by using the `GROUP BY` sql clause.

In [16]:
%%sql
SELECT 
    COUNTRY, 
    AVG(VALUE) AS "Average Energy Produced"
FROM 
    net_electricity_production
GROUP BY 
    COUNTRY
ORDER BY
    "Average Energy Produced" DESC;

 * sqlite:///world_energy_production.db
Done.


COUNTRY,Average Energy Produced
United States,345684.0698335962
India,122832.76040583337
Japan,86426.673305282
Canada,52390.68695096796
Brazil,49802.2165759375
Germany,48989.99915834615
France,44755.72620639104
Korea,44600.57219200639
United Kingdom,27132.415068038466
Mexico,25672.545425961536


### 2. What is the average energy production by each energy product type?

For our second question, to find the average energy produced by energy product type we are going to `SELECT` the `PRODUCT` column and use the `AVG` sql clause on the `VALUE` data column. Since we are trying to find the average energy produced by all energy product types, we are going to use the `world_energy_production` table not the views we created previously.

In [17]:
%%sql
SELECT 
    PRODUCT, 
    AVG(VALUE) AS "Average Energy Production by Energy Product"
FROM 
    world_energy_production
WHERE
    PRODUCT NOT IN (
        "Electricity supplied",
        "Net electricity production",
        "Final consumption"
    )
    AND COUNTRY NOT LIKE '%OECD%'
    AND COUNTRY NOT LIKE '%IEA%'
GROUP BY 
    PRODUCT
ORDER BY 
    "Average Energy Production by Energy Product" DESC;

 * sqlite:///world_energy_production.db
Done.


PRODUCT,Average Energy Production by Energy Product
Non-renewables,17385.036553063
Total combustible fuels,14112.131880687122
Fossil fuels,13534.914339310237
Low carbon,10344.296007089953
Coal,7651.843734438295
Renewables,6421.921059906902
Natural gas,6016.722022496549
Nuclear,5547.833900265679
Hydro,3736.821054701924
Wind,1448.9028199678087


Our output shows us that "Non-renewables", "Total combustible fuels", and "Fossil fuels" are the highest on average energy producers by energy product type.

### 3. Which countries have produced the most energy and which countries have produced the least?

We will start by first `SELECT` the `COUNTRY` data column and use the `SUM` sql clause on the `VALUE` data column to calculate the total energy produced by each country. We will then `GROUP BY` `COUNTRY` and `ORDER BY` "Total Energy Production by Country" in `DESC` order.

In [18]:
%%sql
SELECT 
    COUNTRY, 
    SUM(VALUE) AS "Total Energy Production by Country"
FROM 
    net_electricity_production
GROUP BY 
    COUNTRY
ORDER BY 
    "Total Energy Production by Country" DESC
LIMIT 3;

 * sqlite:///world_energy_production.db
Done.


COUNTRY,Total Energy Production by Country
United States,53926714.894041
Japan,13482561.035623994
India,11791944.998960003


From our query we can see that the United States, Japan, and India are the countries who produced the most energy.

For the second part of our question, we will follow the same sql query from before, but we will change the `ORDER BY` "Total Energy Production by Country" to `ASC` order.

In [19]:
%%sql
SELECT 
    COUNTRY, 
    SUM(VALUE) AS "Total Energy Production by Country"
FROM 
    net_electricity_production
GROUP BY 
    COUNTRY
ORDER BY 
    "Total Energy Production by Country" ASC
LIMIT 3;

 * sqlite:///world_energy_production.db
Done.


COUNTRY,Total Energy Production by Country
Malta,13913.763948000003
Costa Rica,25017.653755000003
Luxembourg,35666.84202499999


Our output shows us that Malta, Costa Rica, and Luxembourg are the countries with the lowest energy produced.

### 4. What is the minimum and maximum energy produced by any country in any given month and year?

To find the minimum energy produced by any country in any given month and year we are going to `SELECT` the `COUNTRY`,`YEAR`,`MONTH_NAME`, and use the `MIN` SQL clause function on the `VALUE` data column. Let's `GROUP BY` and `COUNTRY`,`YEAR`,`MONTH_NAME`. There are two ways to find the minimum energy produced one using the `HAVING` sql clause and another using a simple `ORDER BY` and `LIMIT` sql clauses.

In [20]:
%%sql
SELECT
    COUNTRY,
    YEAR,
    MONTH_NAME,
    MIN(VALUE) AS "Lowest Energy Production"
FROM
    net_electricity_production
GROUP BY
    COUNTRY,
    YEAR, 
    MONTH_NAME
HAVING
    MIN(VALUE) = (
        SELECT MIN(VALUE)
        FROM net_electricity_production
    );


 * sqlite:///world_energy_production.db
Done.


COUNTRY,YEAR,MONTH_NAME,Lowest Energy Production
Malta,2015,November,39.981


In [21]:
%%sql
SELECT
    COUNTRY,
    YEAR,
    MONTH_NAME,
    MIN(VALUE) AS "Lowest Energy Production"
FROM
    net_electricity_production
GROUP BY
    COUNTRY,
    YEAR,
    MONTH_NAME
ORDER BY
    "Lowest Energy Production" ASC
LIMIT 1;

 * sqlite:///world_energy_production.db
Done.


COUNTRY,YEAR,MONTH_NAME,Lowest Energy Production
Malta,2015,November,39.981


We see that the lowest enegry produced by any given country in any given month and year is Malta in November of 2015.

In [22]:
%%sql
SELECT
    COUNTRY,
    YEAR,
    MONTH_NAME,
    MAX(VALUE) AS "Highest Energy Production"
FROM
    net_electricity_production
GROUP BY
    COUNTRY,
    YEAR,
    MONTH_NAME
HAVING
    MAX(VALUE) = (
        SELECT MAX(VALUE)
        FROM net_electricity_production
    );

 * sqlite:///world_energy_production.db
Done.


COUNTRY,YEAR,MONTH_NAME,Highest Energy Production
United States,2022,July,432965.975804


In [23]:
%%sql
SELECT
    COUNTRY,
    YEAR,
    MONTH_NAME,
    MAX(VALUE) AS "Highest Energy Production"
FROM
    net_electricity_production
GROUP BY
    COUNTRY,
    YEAR,
    MONTH_NAME
ORDER BY
    "Highest Energy Production" DESC
LIMIT 1;

 * sqlite:///world_energy_production.db
Done.


COUNTRY,YEAR,MONTH_NAME,Highest Energy Production
United States,2022,July,432965.975804


For the highest energy produced we get the United States in July of 2022.

### 5. Which years had the lowest and highest energy production and demand?

To answer this question we are going to follow a similar process that we have done for other questions. We will `SELECT` the `YEAR` data column and use the `SUM` sql clause function to calculate the total energy production and energy demand by year.

In [24]:
%%sql
SELECT 
    YEAR,
    SUM(VALUE) AS "Total Energy Production by Year"
FROM 
    net_electricity_production
GROUP BY 
    YEAR
ORDER BY
    "Total Energy Production by Year" ASC
LIMIT 3;

 * sqlite:///world_energy_production.db
Done.


YEAR,Total Energy Production by Year
2013,10391693.063000008
2012,10392343.528999995
2011,10398598.024000004


The years with the lowestt energy production are 2013, 2012, and 2011.

In [25]:
%%sql
SELECT 
    YEAR,
    SUM(VALUE) AS "Total Energy Production by Year"
FROM 
    net_electricity_production
GROUP BY 
    YEAR
ORDER BY
    "Total Energy Production by Year" DESC
LIMIT 3;

 * sqlite:///world_energy_production.db
Done.


YEAR,Total Energy Production by Year
2022,13384563.473613016
2021,13284846.391359994
2018,13211383.110575994


The years with the highest energy production are 2022, 2021, and 2018.

In [26]:
%%sql
SELECT 
    YEAR,
    SUM(VALUE) AS "Total Energy Demand by Year"
FROM 
    energy_demand
GROUP BY 
    YEAR
ORDER BY
    "Total Energy Demand by Year" ASC
LIMIT 3;

 * sqlite:///world_energy_production.db
Done.


YEAR,Total Energy Demand by Year
2012,9618431.41899999
2011,9641062.005999997
2013,9645713.755999997


The years with the lowest energy demand are 2012, 2011, and 2013.

In [27]:
%%sql
SELECT 
    YEAR,
    SUM(VALUE) AS "Total Energy Demand by Year"
FROM 
    energy_demand
GROUP BY 
    YEAR
ORDER BY
    "Total Energy Demand by Year" DESC
LIMIT 3;

 * sqlite:///world_energy_production.db
Done.


YEAR,Total Energy Demand by Year
2018,12490092.995364007
2019,12414583.651425006
2020,12149548.372499991


The years with the highest energy demand are 2018, 2019, and 2020.

### 6. How does energy production and demand change from season to season?

For this question we are going to identify the `Season` of each record in our data. To do this we are going to use the `CASE` sql clause to catergorize each record in our data into either Winter, Spring, Summer, Fall or Unknown.

In [28]:
%%sql
SELECT
    CASE
        WHEN MONTH IN (12, 1, 2) THEN 'Winter'
        WHEN MONTH IN (3, 4, 5) THEN 'Spring'
        WHEN MONTH IN (6, 7, 8) THEN 'Summer'
        WHEN MONTH IN (9, 10, 11) THEN 'Fall'
        ELSE 'Unknown'
    END AS Season,
    AVG(VALUE) AS "Average Energy Production"
FROM
    net_electricity_production
GROUP BY
    Season
ORDER BY
    "Average Energy Production" DESC;

 * sqlite:///world_energy_production.db
Done.


Season,Average Energy Production
Summer,24783.664107862623
Winter,24707.871789814388
Fall,22964.991708214948
Spring,22771.674150380313


We see that Summer and Winter are the seasons with the highest energy production.

In [29]:
%%sql
SELECT
    CASE
        WHEN MONTH IN (12, 1, 2) THEN 'Winter'
        WHEN MONTH IN (3, 4, 5) THEN 'Spring'
        WHEN MONTH IN (6, 7, 8) THEN 'Summer'
        WHEN MONTH IN (9, 10, 11) THEN 'Fall'
        ELSE 'Unknown'
    END AS Season,
    AVG(VALUE) AS "Average Energy Demand"
FROM
    energy_demand
GROUP BY
    Season
ORDER BY
    "Average Energy Demand" DESC;

 * sqlite:///world_energy_production.db
Done.


Season,Average Energy Demand
Winter,22867.044367137438
Summer,22848.179884756217
Fall,21261.593288596378
Spring,20974.91436095521


The energy demand is highest in winter and summer seasons on average.

### 7. How has the energy production of renewable energy products changed over time?

For this question we will not be using the views we creater earlier in the project. We will `SELECT` the `YEAR`,`PRODUCT` and use the `SUM` sql clause function on the `VALUE` data column to find the total energy production by year. Additionally, we will use the `WHERE` sql clause to condition on energy product types renewables.

In [30]:
%%sql
SELECT
    YEAR,
    PRODUCT,
    SUM(VALUE) AS "Total Energy Production by Year"
FROM
    world_energy_production
WHERE
    PRODUCT = "Renewables"
GROUP BY
    YEAR;

 * sqlite:///world_energy_production.db
Done.


YEAR,PRODUCT,Total Energy Production by Year
2010,Renewables,7798026.598000009
2011,Renewables,8301214.965000001
2012,Renewables,8722377.841000002
2013,Renewables,9240592.922999997
2014,Renewables,9647251.412000015
2015,Renewables,10777211.974934999
2016,Renewables,11344050.159824984
2017,Renewables,11916530.917441
2018,Renewables,12475469.534208994
2019,Renewables,12918149.488351999


From our query we see that the energy production of renewables increases by each year.

### 8. How did the covid-19 pandemic change energy production and demand?

Let's answer our last question by first using the `SELECT` sql clause on the `YEAR` and use the `AVG` sql clause function on the `VALUE` data column to find the average energy production by year and month. Additionally, we will use the `WHERE` sql clause to condition on the year of 2020 and 2021. Within the `WHERE` sql clause I am going to add a `AND` condition sql clause to get the records in our data where the `YEAR` is between 2019 or 2022.

In [31]:
%%sql
SELECT
    YEAR,
    AVG(VALUE) AS "Average Energy Production by Year and Month"
FROM
    net_electricity_production
WHERE
    YEAR >= 2019
    AND
    YEAR <= 2022
GROUP BY
    YEAR;

 * sqlite:///world_energy_production.db
Done.


YEAR,Average Energy Production by Year and Month
2019,23714.294974855053
2020,23150.14885775905
2021,23554.6921832624
2022,23731.495520590455


In [32]:
%%sql
SELECT
    YEAR,
    AVG(VALUE) AS "Average Energy Demand by Year and Month"
FROM
    energy_demand
WHERE
    YEAR >= 2019
    AND
    YEAR <= 2022
GROUP BY
    YEAR;

 * sqlite:///world_energy_production.db
Done.


YEAR,Average Energy Demand by Year and Month
2019,22490.187774320664
2020,22010.051399456504
2021,17918.92922953014
2022,22557.22503527479


From the output we can see that energy production and energy demand are lower on average in the years of the covid-19 pandemic,2020 and 2021, compared to 2019 and 2022. 